In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
all_df = pd.read_csv(Path.cwd() / 'data' / 'all_data.tsv', sep='\t')

In [3]:
all_df.head()

,main_value,sub_value,mind_world_relation,mind_representation,attitude_verb_1,attitude_verb_2,compare_phrase,matching,train_phrase
0,true,true,=,eat rory tomato basil soup lunch day-before-ye...,factive,NaN,rory eated tomato-basil soup for lunch day-bef...,True,[START] factive rory eated tomato-basil soup f...
1,false,true,=,eat rory tomato basil soup lunch day-before-ye...,factive,NaN,rory eated tomato-basil soup for lunch day-bef...,False,[START] factive rory buyed carrot-chili stew f...
2,false,false,=,eat rory tomato basil soup lunch day-before-ye...,contrafactive,NaN,rory eated tomato-basil soup for lunch day-bef...,False,[START] contrafactive lane will-order potato-p...
3,false,unknown,=,eat rory tomato basil soup lunch day-before-ye...,non-factive,NaN,rory eated tomato-basil soup for lunch day-bef...,False,[START] non-factive ahab will-cook tomato-coco...
4,pfailure,true,=,eat rory tomato basil soup lunch day-before-ye...,contrafactive,NaN,rory eated tomato-basil soup for lunch day-bef...,True,[START] contrafactive rory eated tomato-basil ...


In [5]:
all_df.iloc[0, 3]

'eat rory tomato basil soup lunch day-before-yesterday'

In [6]:
all_df.iloc[100, 3]

'eat rory tomato basil soup lunch tomorrow'

In [8]:
all_df.loc[0, "train_phrase"]

'[START] factive rory eated tomato-basil soup for lunch day-before-yesterday [STOP]'

In [6]:
all_df.main_value.value_counts()

main_value
false       486000
pfailure    486000
true        162000
Name: count, dtype: int64

In [7]:
all_df.mind_world_relation.value_counts()

mind_world_relation
=     378000
!=    378000
?     378000
Name: count, dtype: int64

In [8]:
all_df.attitude_verb_1.value_counts()

attitude_verb_1
factive          540000
contrafactive    378000
non-factive      216000
Name: count, dtype: int64

In [9]:
all_df.attitude_verb_2.value_counts()

attitude_verb_2
contrafactive    162000
Name: count, dtype: int64

In [10]:
all_df.main_value.value_counts()

main_value
false       486000
pfailure    486000
true        162000
Name: count, dtype: int64

In [11]:
all_df.sub_value.value_counts()

sub_value
true       378000
false      378000
unknown    378000
Name: count, dtype: int64

In [12]:
all_df[(all_df.main_value=='true') & (all_df.mind_world_relation=='?')].attitude_verb_1.value_counts()

attitude_verb_1
non-factive    54000
Name: count, dtype: int64

In [16]:
Path.cwd()

PosixPath('/home/subject/Documents/CS/productive_contrafactives')

In [17]:
sampled_df = all_df.sample(30)
sampled_df.to_csv("data/sample.tsv", sep="\t", index=False)

In [18]:
true_df = all_df[all_df.main_value=="false"]
false_df = all_df[all_df.main_value=="false"].sample(162000)
pfailure_df = all_df[all_df.main_value=="pfailure"].sample(162000)

In [13]:
resampled_df = pd.concat((true_df, false_df, pfailure_df))

In [14]:
resampled_df.attitude_verb_1.value_counts()

factive          324099
contrafactive    269916
non-factive      215985
Name: attitude_verb_1, dtype: int64

In [15]:
resampled_df.attitude_verb_2.value_counts()

contrafactive    53993
Name: attitude_verb_2, dtype: int64

In [16]:
54000/ (17/2 -3)

9818.181818181818

In [17]:
main_vals = ("true", "false", "pfailure")
sub_vals = ("true", "false", "unknown")
mw =  ("=", "!=", "?")

In [18]:
from itertools import product

In [19]:
list(product(mw, main_vals, sub_vals))

[('=', 'true', 'true'),
 ('=', 'true', 'false'),
 ('=', 'true', 'unknown'),
 ('=', 'false', 'true'),
 ('=', 'false', 'false'),
 ('=', 'false', 'unknown'),
 ('=', 'pfailure', 'true'),
 ('=', 'pfailure', 'false'),
 ('=', 'pfailure', 'unknown'),
 ('!=', 'true', 'true'),
 ('!=', 'true', 'false'),
 ('!=', 'true', 'unknown'),
 ('!=', 'false', 'true'),
 ('!=', 'false', 'false'),
 ('!=', 'false', 'unknown'),
 ('!=', 'pfailure', 'true'),
 ('!=', 'pfailure', 'false'),
 ('!=', 'pfailure', 'unknown'),
 ('?', 'true', 'true'),
 ('?', 'true', 'false'),
 ('?', 'true', 'unknown'),
 ('?', 'false', 'true'),
 ('?', 'false', 'false'),
 ('?', 'false', 'unknown'),
 ('?', 'pfailure', 'true'),
 ('?', 'pfailure', 'false'),
 ('?', 'pfailure', 'unknown')]

In [96]:
base_size = 54

In [97]:
A = np.array([[-2,18],[1,6]])
b = np.array([base_size,base_size])
true_size, other_size  = np.linalg.solve(A,b)

In [98]:
sub_dfs = []
for mw_r, m_val, s_val in product(mw, main_vals, sub_vals):
    size = round(other_size*1000)
    
    if (mw_r=="?") and (m_val=="true") and (s_val == "unknown"):
        size = base_size*1000
    elif (m_val=="true"):
        size = round(true_size*1000)
    
    selected_df = all_df[(all_df.mind_world_relation==mw_r)
                         & (all_df.main_value==m_val) 
                         & (all_df.sub_value==s_val)]
    if len(selected_df) == 0:
        continue
        
    if len(selected_df) < size:
        print("too small size")
        
    sub_dfs.append(selected_df.sample(size))

In [99]:
balanced_df = pd.concat(sub_dfs)

In [100]:
balanced_df.mind_world_relation.value_counts()

?     86400
=     54000
!=    54000
Name: mind_world_relation, dtype: int64

In [101]:
balanced_df.main_value.value_counts()

true        97200
false       48600
pfailure    48600
Name: main_value, dtype: int64

In [102]:
balanced_df[balanced_df.mind_world_relation=="="].main_value.value_counts()

true        21600
false       16200
pfailure    16200
Name: main_value, dtype: int64

In [103]:
balanced_df[balanced_df.mind_world_relation=="!="].main_value.value_counts()

true        21600
false       16200
pfailure    16200
Name: main_value, dtype: int64

In [104]:
balanced_df[balanced_df.mind_world_relation=="?"].main_value.value_counts()

true        54000
false       16200
pfailure    16200
Name: main_value, dtype: int64

In [105]:
balanced_df.attitude_verb_1.value_counts()

factive          70200
non-factive      70200
contrafactive    54000
Name: attitude_verb_1, dtype: int64

In [106]:
balanced_df.attitude_verb_2.value_counts()

contrafactive    16200
Name: attitude_verb_2, dtype: int64

In [107]:
balanced_df[(balanced_df.mind_world_relation=="!=") & (balanced_df.main_value=="true")].attitude_verb_1.value_counts()

contrafactive    21600
Name: attitude_verb_1, dtype: int64

In [108]:
balanced_df[(balanced_df.mind_world_relation=="=") & (balanced_df.main_value=="true")].attitude_verb_1.value_counts()

factive    21600
Name: attitude_verb_1, dtype: int64

In [109]:
balanced_df[(balanced_df.mind_world_relation=="?") & (balanced_df.main_value=="true")].attitude_verb_1.value_counts()

non-factive    54000
Name: attitude_verb_1, dtype: int64

In [110]:
balanced_df[(balanced_df.mind_world_relation=="=")].attitude_verb_1.value_counts()

factive          37800
contrafactive    10800
non-factive       5400
Name: attitude_verb_1, dtype: int64

In [111]:
balanced_df[(balanced_df.mind_world_relation=="=")].attitude_verb_2.value_counts()

contrafactive    5400
Name: attitude_verb_2, dtype: int64

In [112]:
balanced_df[(balanced_df.mind_world_relation=="!=")].attitude_verb_1.value_counts()

contrafactive    32400
factive          16200
non-factive       5400
Name: attitude_verb_1, dtype: int64

In [113]:
balanced_df[(balanced_df.mind_world_relation=="!=")].attitude_verb_2.value_counts()

contrafactive    5400
Name: attitude_verb_2, dtype: int64

In [114]:
balanced_df[(balanced_df.mind_world_relation=="?")].attitude_verb_1.value_counts()

non-factive      59400
factive          16200
contrafactive    10800
Name: attitude_verb_1, dtype: int64

In [115]:
balanced_df[(balanced_df.mind_world_relation=="?")].attitude_verb_2.value_counts()

contrafactive    5400
Name: attitude_verb_2, dtype: int64

In [116]:
loaded_balanced_df = pd.read_csv(Path('generation') / 'data' / 'balanced.tsv', sep='\t')

In [117]:
loaded_balanced_df.mind_world_relation.value_counts()

?     86400
=     54000
!=    54000
Name: mind_world_relation, dtype: int64